# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [36]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [37]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,16.81,95,100,14.91,SH,1733111594
1,1,hamilton,39.1834,-84.5333,-3.92,69,75,2.68,US,1733111595
2,2,lihue,21.9789,-159.3672,24.77,74,20,4.12,US,1733111596
3,3,qaqortoq,60.7167,-46.0333,-1.66,98,100,2.58,GL,1733111597
4,4,tiksi,71.6872,128.8694,-25.13,100,65,2.16,RU,1733111598


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [38]:
%%capture --no-display

# Configure the map plot
map_plot_1=city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles = "OSM",
    frame_width=700,
    frame_height=500,
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [39]:
# Narrow down cities that fit criteria and drop any results with null values
filter_city_data_df = city_data_df.loc[
    (city_data_df["Max Temp"] > 0) & 
    (city_data_df["Humidity"] < 40) & 
    (city_data_df["Cloudiness"] < 20)
]

# Drop any rows with null values
filter_city_data_df=filter_city_data_df.dropna()

# Display sample data
filter_city_data_df.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
151,151,chandler,33.3062,-111.8412,18.82,36,0,1.54,US,1733111577
165,165,agadez,19.7500,10.2500,13.50,30,6,3.49,NE,1733111794
168,168,illela,13.7292,5.2975,18.47,23,10,4.47,NG,1733111798
175,175,taghazout,30.5426,-9.7112,20.90,39,18,3.57,MA,1733111807
185,185,huaidian,33.4333,115.0333,13.58,32,0,6.16,CN,1733111819
189,189,khedbrahma,24.0333,73.0500,22.48,29,0,2.72,IN,1733111823
192,192,malpura,26.2833,75.3833,21.19,29,0,2.15,IN,1733111827
201,201,las palmas,28.4204,-14.0131,17.44,32,0,2.57,ES,1733111837
214,214,changleng,28.7000,115.8167,20.56,36,4,1.39,CN,1733111852
287,287,ingeniero luis a. huergo,-39.0833,-67.2333,13.27,20,4,3.90,AR,1733111944


### Step 3: Create a new DataFrame called `hotel_df`.

In [41]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
columns = ['City', 'Country', 'Lat', 'Lng', 'Humidity']
hotel_df = filter_city_data_df[columns].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
151,chandler,US,33.3062,-111.8412,36,
165,agadez,NE,19.7500,10.2500,30,
168,illela,NG,13.7292,5.2975,23,
175,taghazout,MA,30.5426,-9.7112,39,
185,huaidian,CN,33.4333,115.0333,32,
189,khedbrahma,IN,24.0333,73.0500,29,
192,malpura,IN,26.2833,75.3833,29,
201,las palmas,ES,28.4204,-14.0131,32,
214,changleng,CN,28.7000,115.8167,36,
287,ingeniero luis a. huergo,AR,-39.0833,-67.2333,20,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [42]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
chandler - nearest hotel: Crowne Plaza - San Marcos Golf Resort
agadez - nearest hotel: No hotel found
illela - nearest hotel: HOTEL KADO
taghazout - nearest hotel: Riad Taghazout
huaidian - nearest hotel: No hotel found
khedbrahma - nearest hotel: No hotel found
malpura - nearest hotel: Tordi Garh
las palmas - nearest hotel: Era de La Corte
changleng - nearest hotel: 前湖酒店
ingeniero luis a. huergo - nearest hotel: No hotel found
lhasa - nearest hotel: 拉萨市香格里拉大酒店
alwar - nearest hotel: Nirvana
rikaze - nearest hotel: Gesar Hotel
las vegas - nearest hotel: Downtown Grand
deqen - nearest hotel: No hotel found
puqi - nearest hotel: No hotel found
jindabyne - nearest hotel: Lake Jindabyne Hotel
phillaur - nearest hotel: Hotel Tri Mohan.
eduardo castex - nearest hotel: Hotel San Cristóbal
vryburg - nearest hotel: International Hotel
faya-largeau - nearest hotel: No hotel found
san agustin de valle fertil - nearest hotel: Hotel Rustico Cerro Del Valle


,City,Country,Lat,Lng,Humidity,Hotel Name
151,chandler,US,33.3062,-111.8412,36,Crowne Plaza - San Marcos Golf Resort
165,agadez,NE,19.7500,10.2500,30,No hotel found
168,illela,NG,13.7292,5.2975,23,HOTEL KADO
175,taghazout,MA,30.5426,-9.7112,39,Riad Taghazout
185,huaidian,CN,33.4333,115.0333,32,No hotel found
189,khedbrahma,IN,24.0333,73.0500,29,No hotel found
192,malpura,IN,26.2833,75.3833,29,Tordi Garh
201,las palmas,ES,28.4204,-14.0131,32,Era de La Corte
214,changleng,CN,28.7000,115.8167,36,前湖酒店
287,ingeniero luis a. huergo,AR,-39.0833,-67.2333,20,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [44]:
%%capture --no-display

# Configure the map plot
hotels_map=hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    color="City",
    size= "Humidity",
    hover_cols= ["Hotel Name", "Country"]
)
    

# Display the map
hotels_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)